# `requests` 통해 웹 정보를 받아 크롤링

In [1]:
import requests

### 테스트

In [2]:
## https://comic.naver.com/webtoon 에 접속했을 때, 요일별 현재 연재 중인 전체 웹툰이 담긴 api
url = "https://comic.naver.com/api/webtoon/titlelist/weekday?order=user"

head  = {'User-Agent':
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'}

r = requests.get(url, headers=head)
# r.text

In [11]:
## 하나의 웹툰 찾기
r.json()['titleListMap']['TUESDAY'][0]

{'titleId': 783053,
 'titleName': '김부장',
 'author': '박태준 만화회사 / 정종택',
 'thumbnailUrl': 'https://image-comic.pstatic.net/webtoon/783053/thumbnail/thumbnail_IMAG21_d7732f14-26da-4e35-8762-660cc87b53f1.jpg',
 'up': False,
 'rest': False,
 'bm': False,
 'adult': False,
 'starScore': 9.47587,
 'viewCount': 0,
 'openToday': False,
 'potenUp': False,
 'bestChallengeLevelUp': False,
 'finish': False,
 'new': False}

In [8]:
### 화요일 웹툰 titleId 모으기 테스트

a = []
for i in range(r.json()['titleListMap']['TUESDAY'].__len__()):
    a.append(r.json()['titleListMap']['TUESDAY'][i]['titleId'])
a[:6]

[783053, 784248, 796152, 790713, 789979, 730656]

In [9]:
r.json()['titleListMap'].keys()

dict_keys(['MONDAY', 'SUNDAY', 'TUESDAY', 'SATURDAY', 'THURSDAY', 'FRIDAY', 'WEDNESDAY'])

## 요일 전체 `titleId` 가져오기

In [3]:
import requests
from tqdm import tqdm

In [7]:
## https://comic.naver.com/webtoon 에 접속했을 때, 요일별 현재 연재 중인 전체 웹툰이 담긴 api
url = "https://comic.naver.com/api/webtoon/titlelist/weekday?order=user"

head  = {'User-Agent':
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'}

r = requests.get(url, headers=head)

In [5]:
week_day = list(r.json()['titleListMap'].keys()) # 요일 담은 리스트

titleId_list =[]
for i in tqdm(week_day):
    for j in range(r.json()['titleListMap'][i].__len__()):
        titleId_list.append(r.json()['titleListMap'][i][j]['titleId'])


100%|██████████| 7/7 [00:01<00:00,  5.83it/s]


In [6]:
len(titleId_list)

622

In [69]:
# r.json()['titleListMap']

In [11]:
# titleId_list
from tqdm import tqdm

In [67]:
# 각 웹툰 ID와 매칭되는 별점 (starScore) 값을 저장.
# 별점은 round()를 통해 소수 둘째 자리 까지 반올림함.
star_score_list = []
titleId_list = []
for day, webtoons in r.json()['titleListMap'].items():
    for i in webtoons :
        star_score_list.append(round(i['starScore'],2))
        titleId_list.append(i['titleId'])
        
# star_score_list

In [27]:
import pickle

In [28]:
### 현재 연재 중인 웹툰의 ID 값을 저장한 pickle 파일
### 매일 연재 웹툰의 개수가 달라
### 이를 저장하여 고정되게 사용할 수 있게 함. (622개의 데이터)
with open('webtoon_id.pkl', 'wb') as f:
    pickle.dump(titleId_list, f)

In [29]:
with open('webtoon_id.pkl', 'rb') as f:
    titleId_list = pickle.load(f)

In [50]:
with open('./webtoon_star_score.pkl', 'wb') as f:
    pickle.dump(star_score_list, f)

In [51]:
with open('./webtoon_star_score.pkl', 'rb') as f:
    ssl = pickle.load(f)

In [53]:
ssl[0]

9.93

In [40]:
len(titleId_list)

621

## 각 웹툰의 정보 크롤링

In [11]:
import requests
import pandas as pd

### 테스트

In [17]:
# 하나의 웹툰에 저장된 정보가 담긴 url api
url2 = "https://comic.naver.com/api/article/list/info?titleId=796075"

head2 = {'User-Agent':
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'}

In [18]:
r2 = requests.get(url2, headers=head2)
# r2.text

In [68]:
r2.json().keys()

dict_keys(['titleId', 'thumbnailUrl', 'posterThumbnailUrl', 'sharedThumbnailUrl', 'titleName', 'webtoonLevelCode', 'author', 'thumbnailBadgeList', 'age', 'publishDescription', 'synopsis', 'favoriteCount', 'favorite', 'firstArticle', 'publishDayOfWeekList', 'rest', 'finished', 'dailyPass', 'curationTagList', 'adBannerList', 'chargeBestChallenge', 'contentsNo', 'new'])

In [82]:
## 모을 데이터 컬럼:
### "titleId, titleName, writer, painter, age , favoriteCount, thumbnailUrl, publishDayOfWeekList, curationTagList"


In [69]:
r2.json()['curationTagList']

[{'id': 796075,
  'tagName': '무협/사극',
  'urlPath': '/webtoon?tab=genre&genre=HISTORICAL',
  'curationType': 'GENRE_HISTORICAL'},
 {'id': 184,
  'tagName': '환골탈태',
  'urlPath': '/curation/list?type=CUSTOM_TAG&id=184',
  'curationType': 'CUSTOM_TAG'},
 {'id': 796075,
  'tagName': '소설원작',
  'urlPath': '/curation/list?type=NOVEL_ORIGIN',
  'curationType': 'NOVEL_ORIGIN'}]

In [19]:
### 만약 분석을 추가한다면?
#### 관심도 수 와 웹툰 랭킹을 연관지을 수 있으려나?? -> 조회순으로 보여지는 각 요일의 순서 == 랭킹이라고 가정.


In [70]:
r2.json()['titleName']

'절대검감'

In [71]:
r2.json()['author']['writers'][0]['name']

'김두루미'

In [72]:
r2.json()['age']['type']

'RATE_12'

In [73]:
r2.json()['favoriteCount']

115387

In [74]:
r2.json()['publishDayOfWeekList'][0]

'MONDAY'

In [21]:
a = []
for i in range(r2.json()['curationTagList'].__len__()):
    a.append(r2.json()['curationTagList'][i]['tagName'])



In [22]:
a
## 해당 리스트를 데이터프레임에서 한 컬럼안에 넣기 위해선 문자열로 바꿔야한다.

['무협/사극', '환골탈태', '소설원작']

In [77]:
## 이후에 잘 추출하기 위해 공백을 없앰.
', '.join(a).replace(" ","")

'무협/사극,환골탈태,소설원작'

In [78]:
b = [
    r2.json()['titleName'],
    r2.json()['author']['writers'][0]['name'],
    r2.json()['age']['type'],
    r2.json()['favoriteCount'],
    r2.json()['publishDayOfWeekList'][0],
    ', '.join(a).replace(" ","")
]
b

['절대검감', '김두루미', 'RATE_12', 115387, 'MONDAY', '무협/사극,환골탈태,소설원작']

In [53]:
c = {
    'Title': [r2.json()['titleName']],
    'Author': [r2.json()['author']['writers'][0]['name']],
    'Age': [r2.json()['age']['type']],
    'Favorite': [r2.json()['favoriteCount']],
    'PublicDay': r2.json()['publishDayOfWeekList'],
    'HashTag': [', '.join(a).replace(" ","")]
}
c

{'Title': ['절대검감'],
 'Author': ['김두루미'],
 'Age': ['RATE_12'],
 'Favorite': [115482],
 'PublicDay': ['MONDAY'],
 'HashTag': ['무협/사극,환골탈태,소설원작']}

In [55]:
m_df=pd.DataFrame(c)

In [57]:
m_df

,Title,Author,Age,Favorite,PublicDay,HashTag
0,절대검감,김두루미,RATE_12,115482,MONDAY,"무협/사극,환골탈태,소설원작"


In [ ]:
## 재귀함수로 데이터프레임을 병합해야함. 아마도? 이건 좀 생각해보자

# 전체 title_id를 통해 데이터프레임 만들기 

In [2]:
import requests
import pandas as pd

In [3]:
# 하나의 웹툰에 저장된 정보가 담긴 url api
### Id 부분은 format으로 동적 변경
detail_url = "https://comic.naver.com/api/article/list/info?titleId=711422"

head = {'User-Agent':
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'}

In [4]:
r2= requests.get(detail_url, headers=head)

In [81]:
r2.json()['publishDayOfWeekList']

['TUESDAY', 'FRIDAY']

In [73]:
r2.json().keys()

dict_keys(['titleId', 'thumbnailUrl', 'posterThumbnailUrl', 'sharedThumbnailUrl', 'titleName', 'webtoonLevelCode', 'author', 'thumbnailBadgeList', 'age', 'publishDescription', 'synopsis', 'favoriteCount', 'favorite', 'firstArticle', 'publishDayOfWeekList', 'rest', 'finished', 'dailyPass', 'curationTagList', 'adBannerList', 'chargeBestChallenge', 'contentsNo', 'new'])

In [84]:
type(r2.json()['favoriteCount'])

int

In [81]:
r2.json()['author']['painters'][0]['name']

'김현수'

In [22]:
r2.json()['thumbnailUrl']

'https://image-comic.pstatic.net/webtoon/774866/thumbnail/thumbnail_IMAG21_46cf2e65-7b7c-4fb7-8f54-0078456d8ff0.jpg'

In [14]:
r2.json()['curationTagList'].__len__()

3

In [15]:
# 'curationTagList' 부분은 딕셔너리 안에 딕셔너리 구조로 되어 있기 때문에
## 이를 각각의 데이터를 뽑아 리스트 안에 넣었고,
### 한 컬럼 안에 해당 데이터들을 모두 넣기 위해
#### ", ".join() 을 통해 문자열로 변환함.
##### 또한 replace() 를 통해 문자열 간의 공백을 없애고, ',' 로만 구분짓게 함.

tmp = []
for i in range(r2.json()['curationTagList'].__len__()):
    tmp.append(r2.json()['curationTagList'][i]['tagName'])

", ".join(tmp).replace(" ","")

'스릴러,2020지상최대공모전,서스펜스'

In [98]:
tmp2 = []
for i in range(r2.json()['publishDayOfWeekList'].__len__()):
    tmp2.append(r2.json()['publishDayOfWeekList'][i])   
days = ", ".join(tmp2).replace(" ","")
days

'TUESDAY,FRIDAY'

In [15]:
import pickle

In [16]:
with open('./webtoon_id.pkl','rb') as f:
    titleId_list = pickle.load(f)

In [13]:
import requests
from tqdm import tqdm
import pandas as pd

In [33]:
detail_url = "https://comic.naver.com/api/article/list/info?titleId={}"

head = {'User-Agent':
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'}

data_list = []

for x in tqdm(titleId_list) :
    r2= requests.get(detail_url.format(x), headers=head)

    # 'curationTagList' 부분은 딕셔너리 안에 딕셔너리 구조로 되어 있기 때문에
    ## 이를 각각의 데이터를 뽑아 리스트 안에 넣었고,
    ### 한 컬럼 안에 해당 데이터들을 모두 넣기 위해
    #### ", ".join() 을 통해 문자열로 변환함.
    ##### 또한 replace() 를 통해 문자열 간의 공백을 없애고, ',' 로만 구분짓게 함.
    tmp = []
    for i in range(r2.json()['curationTagList'].__len__()):
        tmp.append(r2.json()['curationTagList'][i]['tagName'])
        
    tags = ", ".join(tmp).replace(" ","")

    temp = {
        'Id' : x,
        'Title': r2.json()['titleName'],
        'Writer': r2.json()['author']['writers'][0]['name'],
        'Painter' : r2.json()['author']['painters'][0]['name'],
        'Age': r2.json()['age']['type'],
        'Favorite': r2.json()['favoriteCount'],
        'ThumbnailUrl' :  r2.json()['thumbnailUrl'],
        'PublishDay': r2.json()['publishDayOfWeekList'],
        'HashTags': tags
    }
    data_list.append(temp)




100%|██████████| 622/622 [00:27<00:00, 22.47it/s]


In [18]:
tags

'무협/사극'

In [100]:
pd.DataFrame(data_list).sort_values('Favorite',ascending=False).head()

,Id,Title,Writer,Painter,Age,Favorite,ThumbnailUrl,PublishDay,HashTags
383,641253,외모지상주의,박태준,박태준,RATE_15,2361336,https://image-comic.pstatic.net/webtoon/641253...,FRIDAY,"드라마,학원물,먼치킨"
224,183559,신의 탑,SIU,SIU,RATE_12,1965213,https://image-comic.pstatic.net/webtoon/183559...,MONDAY,"판타지,세계관,이능력배틀물,소년왕도물,먼치킨"
372,597447,프리드로우,전선욱,전선욱,RATE_15,1631773,https://image-comic.pstatic.net/webtoon/597447...,SATURDAY,"드라마,힘숨찐,학원물,성장드라마"
251,602910,윈드브레이커,조용석,조용석,RATE_12,1149312,https://image-comic.pstatic.net/webtoon/602910...,MONDAY,"스포츠,스포츠성장"
267,769209,화산귀환,LICO,LICO,RATE_15,1137254,https://image-comic.pstatic.net/webtoon/769209...,WEDNESDAY,"무협/사극,세계관,동양풍판타지,회귀,힘숨찐,이세계,먼치킨,소설원작"


In [1]:
r2.json()['StarScore']

NameError: name 'r2' is not defined

In [119]:
pd.DataFrame(data_list).to_csv("./webtoon.csv", encoding="utf-8-sig", index=False)

In [2]:
import pandas as pd

In [ ]:
#### doc string 활용 내용

## 컬럼 설명
# Id 웹툰 id 값
# Title 웹툰 제목
# Writer 글 작가
# Painter 그림 작가
# Age 시청 연령
# Favorite 관심도 수
# ThumbnailUrl 썸네일 사진 링크
# PublishDay 연재 요일
# HashTags 웹툰 내용 관련 해시태그

In [56]:
with open('./webtoon_id.pkl', 'rb') as f:
    webtoon_ids = pickle.load(f)

In [60]:
# webtoon_ids

In [59]:
with open('./webtoon_star_score.pkl', 'rb') as f:
    stars = pickle.load(f)

In [64]:
# 각 웹툰의 상세 내용이 담긴 API 주소값
detail_url = "https://comic.naver.com/api/article/list/info?titleId={}"

data_list = []
for k in tqdm(range(len(webtoon_ids))) :
        x = webtoon_ids[k]
        
        r2= requests.get(detail_url.format(x), headers=head)

        # 해시태그 값들을 저장하기 위한 빈 리스트
        
        # 'curationTagList' 부분은 딕셔너리 안에 딕셔너리 구조로 되어 있기 때문에
        ## 이를 각각의 데이터를 뽑아 리스트 안에 넣었고,
        ### 한 컬럼 안에 해당 데이터들을 모두 넣기 위해
        #### ", ".join() 을 통해 문자열로 변환함.
        ##### 또한 replace() 를 통해 문자열 간의 공백을 없애고, ',' 로만 구분짓게 함.
        tmp = [] 
        for i in range(r2.json()['curationTagList'].__len__()):
            tmp.append(r2.json()['curationTagList'][i]['tagName'])
        tags = ", ".join(tmp).replace(" ","")

        # 해시태그와 같은 이유로 구현.
        ## 일주일에 두 번이상 연재하는 웹툰이 있긴 때문에 컬럼 값에 해당 내용을 모두 저장하기 위해서
        tmp2 = []
        for i in range(r2.json()['publishDayOfWeekList'].__len__()):
            tmp2.append(r2.json()['publishDayOfWeekList'][i])   
        days = ", ".join(tmp2).replace(" ","")

        # 최종 데이터프레임에 저장될 각각의 값들에 해당하는 dict 변수.
        temp = {
            'WebtoonId' : x,
            'Title': r2.json()['titleName'],
            'Writer': r2.json()['author']['writers'][0]['name'],
            'Painter' : r2.json()['author']['painters'][0]['name'],
            'Age': r2.json()['age']['type'],
            'Favorite': r2.json()['favoriteCount'],
            'StarScore': stars[k],
            'ThumbnailUrl' :  r2.json()['thumbnailUrl'],
            'PublishDay': days,
            'HashTags': tags
        }
        
        
        data_list.append(temp)


100%|██████████| 621/621 [00:58<00:00, 10.71it/s]


In [66]:
    
df = pd.DataFrame(data_list)
# df

## MySQL DB 테이블 만들고 데이터 삽입

In [61]:
import pymysql

In [69]:
try:
    con = pymysql.connect(host='127.0.0.1', user='root', password='8489',  db='encore', charset='utf8')
    cur = con.cursor()
except Exception as e:
    print (e)    

In [77]:

check = cur.execute("show tables like 'webtoon'")
check == True

True

In [63]:
cr_table = """CREATE TABLE webtoon(
    Id INT(11) NOT NULL PRIMARY KEY,
    Title VARCHAR(255),
    Writer VARCHAR(255),
    Painter VARCHAR(255),
    Age VARCHAR(255),
    Favorite INT(11),
    ThumbnailUrl VARCHAR(255),
    PublishDay VARCHAR(255),
    HashTags VARCHAR(255)
)
"""

In [64]:
cur.execute(cr_table)

0

In [65]:
cur.execute("show tables")

9

In [66]:
sql = "INSERT INTO webtoon VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s)"

In [67]:
import csv

In [68]:
with open("./webtoon.csv", "r", encoding='utf-8-sig') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    for idx,row in tqdm(enumerate(spamreader)):
        # print(idx,', '.join(row))
        if idx > 0:
            try:
                cur.execute(sql,row)
            except Exception as e:
                print(e)

623it [00:00, 6498.59it/s]

(1062, "Duplicate entry '800312' for key 'PRIMARY'")
(1062, "Duplicate entry '811721' for key 'PRIMARY'")
(1062, "Duplicate entry '812851' for key 'PRIMARY'")
(1062, "Duplicate entry '711422' for key 'PRIMARY'")
(1062, "Duplicate entry '814048' for key 'PRIMARY'")
(1062, "Duplicate entry '812168' for key 'PRIMARY'")
(1062, "Duplicate entry '801589' for key 'PRIMARY'")
(1062, "Duplicate entry '812354' for key 'PRIMARY'")
(1062, "Duplicate entry '616239' for key 'PRIMARY'")
(1062, "Duplicate entry '802039' for key 'PRIMARY'")
(1062, "Duplicate entry '799267' for key 'PRIMARY'")
(1062, "Duplicate entry '814541' for key 'PRIMARY'")
(1062, "Duplicate entry '804333' for key 'PRIMARY'")
(1062, "Duplicate entry '814398' for key 'PRIMARY'")
(1062, "Duplicate entry '790713' for key 'PRIMARY'")
(1062, "Duplicate entry '800506' for key 'PRIMARY'")


In [57]:
row

['814149',
 '집착광공 집사전락',
 '마자',
 '마자',
 'ALL',
 '13197',
 'https://image-comic.pstatic.net/webtoon/814149/thumbnail/thumbnail_IMAG21_1cf527a2-606b-410a-b74c-b17bf938491e.jpg',
 'SUNDAY',
 '개그,집착물']

In [49]:
import csv
with open('webtoon.csv', newline='', encoding='utf-8-sig') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    for row in spamreader:
        print(', '.join(row))

Id, Title, Writer, Painter, Age, Favorite, ThumbnailUrl, PublishDay, HashTags
762279, 정글쥬스, 형은, 쥬더, RATE_15, 382415, https://image-comic.pstatic.net/webtoon/762279/thumbnail/thumbnail_IMAG21_715e8b9b-f6e7-48fc-af26-88fdf50ff7c5.jpg, THURSDAY, 액션,슈퍼스트링,이능력배틀물,먼치킨
799503, 간첩 18세, 팀꿀빨, 김주인, RATE_15, 87773, https://image-comic.pstatic.net/webtoon/799503/thumbnail/thumbnail_IMAG21_34f3cf97-e658-48bd-80c2-ca4397b1ecfd.jpg, SATURDAY, 액션,학원물
808265, 방송은 방송으로 봐, 박은혁, 박은혁, RATE_15, 32142, https://image-comic.pstatic.net/webtoon/808265/thumbnail/thumbnail_IMAG21_9efa88c9-cd85-41c2-8c4c-8dea39c6acd9.jpg, SATURDAY, 드라마
776668, 마왕까지 한 걸음, 윤홍, 윤홍, RATE_15, 153666, https://image-comic.pstatic.net/webtoon/776668/thumbnail/thumbnail_IMAG21_82e1a5b6-754c-425b-af4b-e89382f35997.jpg, THURSDAY, 판타지,힘숨찐
797468, 누나! 나 무서워, 디귿 , 디귿 , RATE_15, 72950, https://image-comic.pstatic.net/webtoon/797468/thumbnail/thumbnail_IMAG21_31b5356a-65c6-4ae3-a8af-92b1c959357f.jpg, THURSDAY, 스릴러,4차원,학원물,괴담
808508, 줄리에게, 제야, 제야, 

In [52]:
cur.execute(sql, row)

1

In [54]:
row

['814149',
 '집착광공 집사전락',
 '마자',
 '마자',
 'ALL',
 '13197',
 'https://image-comic.pstatic.net/webtoon/814149/thumbnail/thumbnail_IMAG21_1cf527a2-606b-410a-b74c-b17bf938491e.jpg',
 'SUNDAY',
 '개그,집착물']

In [60]:
con.commit()

#### (사용X) `selenium` 을 활용한 크롤링

In [ ]:
# pip install selenium

In [ ]:
from selenium import webdriver
import selenium

In [ ]:
driver = webdriver.Chrome()

In [ ]:
driver.get("https://comic.naver.com/webtoon")

In [ ]:
from selenium.webdriver.common.by import By
driver.find_element(By.CSS_SELECTOR,
                    "#container > div.component_wrap.type2 > div.WeekdayMainView__daily_all_wrap--UvRFc > div:nth-child(1) > ul > li:nth-child(1) > div > a > span").click()

In [ ]:
driver.find_element(By.CSS_SELECTOR,
).click()